In [21]:
import json
import subprocess
import time
import statistics

In [2]:
# query anchors from the city list. This code can be skipped if you have city_ping_results.json file.

anchors_by_city = {}

with open('../../info/30-largest-cities-by-population-2026.json', 'r') as file_cities:
    data = json.load(file_cities)
    for entry in data:
        anchors = []

        city = entry['city']
        city_encoded = city.replace(' ', '%20')
        url = f'https://atlas.ripe.net/api/v2/anchors?search={city_encoded}'

        cp = subprocess.run(
            ['curl', '-s', url],
            capture_output=True,
            text=True,
            check=True
        )
        curl_result = json.loads(cp.stdout.strip())
        for anchor_entry in curl_result['results']:
            if not 'ip_v4' in anchor_entry:
                continue
            if anchor_entry['ip_v4'] in [None, '']:
                continue
            if anchor_entry['is_disabled'] == True:
                continue
            if anchor_entry['date_decommissioned'] != None:
                continue

            anchors.append(anchor_entry)    

        anchors_by_city[city] = anchors
        print(f'{city}: {len(anchors)}')


Tokyo: 15
Jakarta: 7
Delhi: 1
Guangzhou: 0
Mumbai: 2
Manila: 1
Shanghai: 0
São Paulo: 2
Seoul: 4
Mexico City: 1
Kolkata: 0
New York City: 3
Chengdu: 0
Cairo: 0
Karachi: 0
Dhaka: 1
Beijing: 0
Bangkok: 1
Shenzhen: 0
Moscow: 10
Buenos Aires: 2
Lagos: 1
Los Angeles: 13
Ho Chi Minh City: 3
Osaka: 2
Johannesburg: 7
Istanbul: 8
Tehran: 7
Kinshasa: 0
Rio de Janeiro: 0


In [3]:
city_by_anchor_probe = {}

for city, anchors in anchors_by_city.items():
    for anchor in anchors:
        city_by_anchor_probe[anchor['probe']] = city

In [4]:
city_measurements = {} #dict[city]dict[fqdn]measurement

for city, anchors in anchors_by_city.items():
    anchor_measurements = {} #key: fqdn

    for anchor in anchors:
        measurements = []

        #print(anchor)
        url = f'https://atlas.ripe.net/api/v2/measurements/?target={anchor['fqdn']}&tags=anchoring,mesh&type=ping&status=Ongoing&is_public=True'
        cp = subprocess.run(
            ['curl', '-s', url],
            capture_output=True,
            text=True,
            check=True
        )
        curl_result = json.loads(cp.stdout.strip())

        for measurement in curl_result['results']:
            if ':' in measurement['target_ip']: #ipv6
                continue

            measurements.append(measurement)

        anchor_measurements[anchor['fqdn']] = measurements
        print(f'{anchor['fqdn']}: {len(measurements)}')
    
    city_measurements[city] = anchor_measurements


jp-tyo-as141682-client.anchors.atlas.ripe.net: 1
jp-tyo-as142616.anchors.atlas.ripe.net: 1
jp-tyo-as15133.anchors.atlas.ripe.net: 2
jp-tyo-as151420.anchors.atlas.ripe.net: 1
jp-tyo-as199524.anchors.atlas.ripe.net: 1
jp-tyo-as20473.anchors.atlas.ripe.net: 1
jp-tyo-as2497.anchors.atlas.ripe.net: 1
jp-tyo-as3491.anchors.atlas.ripe.net: 1
jp-tyo-as38649.anchors.atlas.ripe.net: 1
jp-tyo-as45679.anchors.atlas.ripe.net: 0
jp-tyo-as49544.anchors.atlas.ripe.net: 1
jp-tyo-as55569.anchors.atlas.ripe.net: 1
jp-tyo-as59105.anchors.atlas.ripe.net: 1
jp-tyo-as63781.anchors.atlas.ripe.net: 1
jp-tyo-as63834.anchors.atlas.ripe.net: 1
id-jkt-as133800.anchors.atlas.ripe.net: 1
id-jkt-as141640.anchors.atlas.ripe.net: 1
id-jkt-as153112.anchors.atlas.ripe.net: 1
id-jkt-as24523.anchors.atlas.ripe.net: 1
id-jkt-as45701.anchors.atlas.ripe.net: 1
id-jkt-as45725.anchors.atlas.ripe.net: 1
id-jkt-as64300.anchors.atlas.ripe.net: 1
in-del-as12008.anchors.atlas.ripe.net: 1
in-bom-as12008.anchors.atlas.ripe.net: 1
in-n

In [5]:
time_stop = int(time.time())
time_start = time_stop - 5 * 60 * 60 #an hour
print(f'start:{time_start}, stop:{time_stop}')

start:1770552268, stop:1770570268


In [6]:
# for each measurement, we query every other city anchor's probe
all_probes_str = ','.join(map(str, city_by_anchor_probe.keys()))

In [7]:
city_ping_results = {} # dict['city|target']dict['probe']list[rtt]

for city, city_entry in city_measurements.items():
    for fqdn, measurements in city_entry.items():
        for measurement in measurements:
            city_msm_id = f'{city}|{measurement['target']}'
            print(city_msm_id)

            ping_results = {}
            for probe_id in city_by_anchor_probe.keys():
                ping_results[probe_id] = []

            url = f'{measurement['result']}?probe_ids={all_probes_str}&start={time_start}&stop={time_stop}&format=json'
            cp = subprocess.run(
                ['curl', '-s', url],
                capture_output=True,
                text=True,
                check=True
            )
            curl_result = json.loads(cp.stdout.strip())

            for result_entry in curl_result:
                #print(result_entry)
                for trips in result_entry['result']:
                    rtt = trips.get('rtt')
                    if rtt == None:
                        continue
                    try:
                        rtt_float = float(rtt)
                        ping_results[result_entry['prb_id']].append(rtt_float)
                    finally:
                        pass

            city_ping_results[city_msm_id] = ping_results
            

Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net
Tokyo|jp-tyo-as142616.anchors.atlas.ripe.net
Tokyo|jp-tyo-as15133.anchors.atlas.ripe.net
Tokyo|jp-tyo-as15133.anchors.atlas.ripe.net
Tokyo|jp-tyo-as151420.anchors.atlas.ripe.net
Tokyo|jp-tyo-as199524.anchors.atlas.ripe.net
Tokyo|jp-tyo-as20473.anchors.atlas.ripe.net
Tokyo|jp-tyo-as2497.anchors.atlas.ripe.net
Tokyo|jp-tyo-as3491.anchors.atlas.ripe.net
Tokyo|jp-tyo-as38649.anchors.atlas.ripe.net
Tokyo|jp-tyo-as49544.anchors.atlas.ripe.net
Tokyo|jp-tyo-as55569.anchors.atlas.ripe.net
Tokyo|jp-tyo-as59105.anchors.atlas.ripe.net
Tokyo|jp-tyo-as63781.anchors.atlas.ripe.net
Tokyo|jp-tyo-as63834.anchors.atlas.ripe.net
Jakarta|id-jkt-as133800.anchors.atlas.ripe.net
Jakarta|id-jkt-as141640.anchors.atlas.ripe.net
Jakarta|id-jkt-as153112.anchors.atlas.ripe.net
Jakarta|id-jkt-as24523.anchors.atlas.ripe.net
Jakarta|id-jkt-as45701.anchors.atlas.ripe.net
Jakarta|id-jkt-as45725.anchors.atlas.ripe.net
Jakarta|id-jkt-as64300.anchors.atlas.ripe.net
Delhi|

In [8]:
with open('./city_ping_results.json', 'w') as f_city_ping_result_dump:
    json.dump(city_ping_results, f_city_ping_result_dump)

In [ ]:
######## This block should be skipped if you already ran the previous codes in this session. ########
city_ping_results = {}

with open('./city_ping_results.json', 'r') as f_city_ping_result_dump:
    city_ping_results = json.load(f_city_ping_result_dump)
#####################################################################################################

In [9]:
def join_city_name(city1:str, city2:str) -> str:
    return f'{city1}|{city2}' if city1 < city2 else f'{city2}|{city1}'

inter_city_rtts = {}
for city1 in city_by_anchor_probe.values():
    for city2 in city_by_anchor_probe.values():
        inter_city_rtts[join_city_name(city1, city2)] = []

for city_anchor_name, city_entry in city_ping_results.items():
    for probe_id, rtts in city_entry.items():
        if len(rtts) == 0:
            continue

        probe_city = city_by_anchor_probe[probe_id]
        cities = join_city_name(city_anchor_name.split('|')[0], probe_city)

        print(f'{city_anchor_name}:{probe_id}')
        inter_city_rtts[cities].append(rtts)

Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7663
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7039
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6631
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6455
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6425
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7080
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7689
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7563
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7642
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6974
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6370
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6580
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:6681
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7441
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7423
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7284
Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net:7275
Tokyo|jp-tyo-as141682-client.an

In [10]:
inter_city_rtt_between_best_probes = {}
for cities, rtts in inter_city_rtts.items():
    if len(rtts) == 0:
        continue

    print(f'{cities}')
    inter_city_rtt_between_best_probes[cities] = min(rtts, key=min)

Tokyo|Tokyo
Jakarta|Tokyo
Delhi|Tokyo
Mumbai|Tokyo
Manila|Tokyo
São Paulo|Tokyo
Seoul|Tokyo
Mexico City|Tokyo
New York City|Tokyo
Bangkok|Tokyo
Moscow|Tokyo
Buenos Aires|Tokyo
Lagos|Tokyo
Los Angeles|Tokyo
Ho Chi Minh City|Tokyo
Osaka|Tokyo
Johannesburg|Tokyo
Istanbul|Tokyo
Tehran|Tokyo
Jakarta|Jakarta
Delhi|Jakarta
Jakarta|Mumbai
Jakarta|Manila
Jakarta|São Paulo
Jakarta|Seoul
Jakarta|Mexico City
Jakarta|New York City
Bangkok|Jakarta
Jakarta|Moscow
Buenos Aires|Jakarta
Jakarta|Lagos
Jakarta|Los Angeles
Ho Chi Minh City|Jakarta
Jakarta|Osaka
Jakarta|Johannesburg
Istanbul|Jakarta
Jakarta|Tehran
Delhi|Delhi
Delhi|Mumbai
Delhi|Manila
Delhi|São Paulo
Delhi|Seoul
Delhi|Mexico City
Delhi|New York City
Bangkok|Delhi
Delhi|Moscow
Buenos Aires|Delhi
Delhi|Lagos
Delhi|Los Angeles
Delhi|Ho Chi Minh City
Delhi|Osaka
Delhi|Johannesburg
Delhi|Istanbul
Delhi|Tehran
Mumbai|Mumbai
Manila|Mumbai
Mumbai|São Paulo
Mumbai|Seoul
Mexico City|Mumbai
Mumbai|New York City
Bangkok|Mumbai
Moscow|Mumbai
Buenos Aire

In [11]:
with open('./inter_city_rtt_between_best_probes.json', 'w') as f_inter_city_rtt_between_best_probes_dump:
    json.dump(inter_city_rtt_between_best_probes, f_inter_city_rtt_between_best_probes_dump)

In [17]:
valid_cities = {}
for inter_city in inter_city_rtt_between_best_probes.keys():
    split = inter_city.split('|')
    valid_cities[split[0]] = 0
    valid_cities[split[1]] = 0

valid_cities = list(valid_cities.keys())


In [22]:
network_stats = {}

for inter_city, rtts in inter_city_rtt_between_best_probes.items():
    network_stats[inter_city] = {'mean': sum(rtts) / len(rtts), 'stddev': statistics.stdev(rtts)}

In [23]:
print(network_stats)

{'Tokyo|Tokyo': {'mean': 0.10284705777777778, 'stddev': 0.08129137345069962}, 'Jakarta|Tokyo': {'mean': 77.55211876444444, 'stddev': 0.8615256941073047}, 'Delhi|Tokyo': {'mean': 138.6214586177778, 'stddev': 5.543024936526808}, 'Mumbai|Tokyo': {'mean': 140.76996632, 'stddev': 24.774737457130236}, 'Manila|Tokyo': {'mean': 53.99871724888889, 'stddev': 0.13371124900178102}, 'São Paulo|Tokyo': {'mean': 264.6852946133333, 'stddev': 3.627244922699864}, 'Seoul|Tokyo': {'mean': 29.668585075555555, 'stddev': 0.2261705709711751}, 'Mexico City|Tokyo': {'mean': 140.53358322222223, 'stddev': 3.0940706565214295}, 'New York City|Tokyo': {'mean': 149.32642742666667, 'stddev': 1.9684365019613341}, 'Bangkok|Tokyo': {'mean': 84.76010074666668, 'stddev': 5.695541100242792}, 'Moscow|Tokyo': {'mean': 121.87138188444445, 'stddev': 2.802845416436775}, 'Buenos Aires|Tokyo': {'mean': 234.39200398666668, 'stddev': 3.094059137128053}, 'Lagos|Tokyo': {'mean': 318.90027634666666, 'stddev': 0.17285002262610086}, 'Los

In [25]:
with open('./network_stats.json', 'w') as f_network_stats:
    json.dump(network_stats, f_network_stats, indent=4)